This notebook contains an exploration of the Mission Assignment (MA) dataset. We will look for missiing data, obvious connections between features, and connections between particular Emergency Support Functions (ESF).

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from highcharts_core.chart import Chart
from highcharts_core.options import HighchartsOptions
from highcharts_core.options.series.dependencywheel import DependencyWheelSeries
from highcharts_core.options.plot_options.dependencywheel import DependencyWheelOptions
from itertools import combinations
from collections import Counter
import plotly.graph_objects as go
from matplotlib import colors


Adjust Pandas' output settings

In [35]:
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 100)

Load and explore data

In [36]:
df_load = pd.read_parquet('mission_assignments.parquet')
print(df_load.shape)
df_load.head()

(40340, 39)


,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
0,2024081901,R1_Severe Storm-Flooding_08182024,Severe Storm,3612.0,EM,"SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSL...",3612EMCTDOI-USGS01,0,476360,FOS,5.0,High,1,CT,DOI-USGS,DOI-USGS (GEOLOGICAL SURVEY),Stafford Act,2024-08-22T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,2024-08-23T00:00:00.000Z,20000.0,0.0,1.0,0.0,20000.0,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,0.0,1.0,0.0,20000.0,High,USGS Field measurements of flood-water heights...,"As directed by and in coordination with FEMA, ...",ecbcf24e-7d9e-4ced-ae57-4871181703e1,497d57a5e4d071da1721548476b50b64a7321bdd,2024-08-27T18:24:53.439Z
1,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901USDA-APH01,0,475083,FOS,11.0,High,2,PR,USDA-APH,USDA-APHIS (ANIM/PLANT HEALTH,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,29184.0,0.0,1.0,0.0,29184.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,29184.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",72dc30d3-d804-4005-897f-91395be284b2,e0e65997764dc0d660e5b941a18d2f18adabaa55,2024-08-27T18:24:53.439Z
2,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901DOT01,0,475067,FOS,1.0,High,2,PR,DOT,DOT (DEPT OF TRANSPORT-S/E),Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,25000.0,0.0,1.0,0.0,25000.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,25000.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",61a653ea-688c-4218-bef2-662e1c8b7fde,dbb5a30a3fbd2c96318ba12decb71e63ac072b76,2024-08-27T18:24:53.439Z
3,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901DOJ-ATF01,0,475098,FOS,13.0,High,2,PR,DOJ-ATF,"DOJ-ATF (ALCOHOL,TOBACCO,FIRE",Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,73500.0,0.0,1.0,0.0,73500.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,73500.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",eeafed44-bdfa-45be-a2ec-6aeb4199043d,4f49b6e6d98634659a77d554c06b8fdeac4503cd,2024-08-27T18:24:53.439Z
4,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,NaN,SU,None,PR24080901DOE-OE01,0,475064,FOS,12.0,High,2,PR,DOE-OE,DOE-OE (ELEC DELIVERY / ENERG,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,124822.0,0.0,1.0,0.0,124822.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,124822.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",bdb7935b-5df5-4124-87d8-ddd8feb4a8e3,35d63a525dff8c4a93b83d676d303b6089c0373a,2024-08-27T18:24:53.439Z


In [37]:
# Show only columns missing data as there are too many to show in one cell
missing_data = df_load.isna().sum()
missing_data_ = missing_data[missing_data>0]
missing_data_

disasterNumber         7073
declarationTitle       7073
supportFunction        7508
dateRequired           1853
dateObligated             2
assistanceRequested       2
dtype: int64

Perform a quick review of rows that are missing particular column values.

In [38]:
df_load[df_load['disasterNumber'].isna()].sample(5)

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
8814,2020030901,COVID-19 Pandemic,Biological,NaN,SU,None,DC20030901DOD17,3,342827,FOS,0.0,High,3,DC,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2021-02-18T00:00:00.000Z,2021-02-18T00:00:00.000Z,2020-12-01T00:00:00.000Z,2021-04-01T00:00:00.000Z,2021-02-18T00:00:00.000Z,0.0,0.0,1.0,0.0,0.0,2020-12-01T00:00:00.000Z,2021-12-06T00:00:00.000Z,0.0,1.0,0.0,0.0,High,Amendment #3: Extend end date to 04/01/2021 wi...,"As directed by and in coordination with FEMA, ...",8e25068a-724b-4a5c-a9df-6f2301500c0b,3773c9af47c32833ce89ecb624e7a87db6e05a2d,2024-08-27T18:24:53.439Z
17065,2019082401,Hurricane DORIAN,Hurricane,NaN,SU,None,VA19082401GSA02,0,298476,FOS,7.0,High,3,VA,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2019-09-03T00:00:00.000Z,2019-09-02T00:00:00.000Z,2019-09-02T00:00:00.000Z,2019-09-10T00:00:00.000Z,2019-09-03T00:00:00.000Z,10000.0,0.0,1.0,0.0,10000.0,2019-09-02T00:00:00.000Z,2019-09-10T00:00:00.000Z,0.0,1.0,0.0,10000.0,High,Activate GSA to perform duties in support of d...,Activate the appropriate agency personnel to p...,75660d51-38cc-405f-a831-45e8f7065fc0,97eca1de10216b796b93dd7eef561e015ad320b9,2024-08-27T18:24:53.439Z
718,2023090401,Tropical Cyclone Lee,Hurricane,NaN,SU,None,MA23090401DOT01,0,447266,FOS,1.0,Normal,1,MA,DOT,DOT (DEPT OF TRANSPORT-S/E),Stafford Act,2023-09-13T00:00:00.000Z,2023-09-14T00:00:00.000Z,2023-09-14T00:00:00.000Z,2023-09-20T00:00:00.000Z,2023-09-13T00:00:00.000Z,12818.0,0.0,1.0,0.0,12818.0,2023-09-14T00:00:00.000Z,2023-09-20T00:00:00.000Z,0.0,1.0,0.0,12818.0,Normal,"Activate DOT to RRCC, IOF, JFO, or other regio...","As directed by and in coordination with FEMA, ...",1b1588d5-aff1-48b9-bd7d-6dbc4ed8f0a9,e5326978758cc8c32bb1b1832c25fef14c83ca43,2024-08-27T18:24:53.439Z
6886,2020100401,"DO NOT USE TropWeath Invest 92L(October 4, 2020)",Hurricane,NaN,SU,None,GA20100401DOD01,0,332711,FOS,0.0,High,4,GA,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2020-10-06T00:00:00.000Z,2020-10-07T00:00:00.000Z,2020-10-07T00:00:00.000Z,2020-10-14T00:00:00.000Z,2020-10-06T00:00:00.000Z,10000.0,0.0,1.0,0.0,10000.0,2020-10-07T00:00:00.000Z,2020-10-14T00:00:00.000Z,0.0,1.0,0.0,10000.0,High,Provide a Defense Coordinating Officer/Element...,"As requested by and in coordination with FEMA,...",03cc4798-d96e-4856-9025-f37110e81192,e2500fc5abd586fd910cc90a64ad5e4e3f64e50d,2024-08-27T18:24:53.439Z
4569,2022040701,Puerto Rico Power Outage - 4/6/2022,Other,NaN,SU,None,PR22040701COE-SAD02,1,395516,FOS,3.0,High,2,PR,COE-SAD,COE SOUTH ATLANTIC DIVISION,Stafford Act,2022-04-28T00:00:00.000Z,2022-04-28T00:00:00.000Z,2022-04-08T00:00:00.000Z,2022-04-15T00:00:00.000Z,2022-05-03T00:00:00.000Z,-3480000.0,0.0,1.0,0.0,-3480000.0,2022-04-08T00:00:00.000Z,2022-04-15T00:00:00.000Z,0.0,1.0,0.0,-3480000.0,High,"Amendment #1: De-obligate $3,480,000.00 per US...","As directed by and in coordination with FEMA, ...",2abc4e4b-3443-4563-91e9-21ad374299d0,4f891bc1bf3ba4fec5bcb5b76fd45d7bf5de2355,2024-08-27T18:24:53.439Z


In [39]:
df_load[df_load['supportFunction'].isna()].sample(5)

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
35538,2017083101,Hurricane IRMA,Hurricane,3385.0,EM,HURRICANE IRMA,3385EMFLCOE-SAD07,2,239306,FOS,NaN,High,4,FL,COE-SAD,COE SOUTH ATLANTIC DIVISION,Stafford Act,2017-09-12T00:00:00.000Z,2017-09-12T00:00:00.000Z,2017-09-09T00:00:00.000Z,2017-09-29T00:00:00.000Z,2017-09-12T00:00:00.000Z,-3100000.00,0.0,1.0,0.0,-3100000.00,2017-09-09T00:00:00.000Z,2017-09-29T00:00:00.000Z,0.0,1.0,0.0,-3100000.00,High,AMENDMENT #2 - DE-OBLIGATE FUNDS PER GREG JAME...,Activate and employ USACE Debris PRT managemen...,03ba2ab1-d897-4e14-8f1e-05131f89a03e,bfd8193a3d6692c28f2d25a89ed91d04711f3e59,2024-08-27T18:04:54.403Z
37798,2014101401,Tropical Storm ANA,Other,NaN,SU,None,HI14101401DOD01,1,188079,FOS,NaN,High,9,HI,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2015-04-08T00:00:00.000Z,2015-04-08T00:00:00.000Z,2014-10-16T00:00:00.000Z,2014-10-22T00:00:00.000Z,2015-04-30T00:00:00.000Z,-8772.03,0.0,1.0,0.0,-8772.03,2014-10-16T00:00:00.000Z,2014-10-22T00:00:00.000Z,0.0,1.0,0.0,-8772.03,High,Amendment #1: DoD Memo states that mission ma...,"As requested by and in coordination with FEMA,...",8476c230-5ca2-45bf-9d5e-e92b278dd9cc,3a13e37f54ec1595a8767f416397369022852b6d,2024-08-27T18:04:54.403Z
34235,2017091701,Hurricane Maria,Hurricane,4339.0,DR,HURRICANE MARIA,4339DRPRDOD13,0,241322,FOS,NaN,Life Sustaining,2,PR,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2017-09-22T00:00:00.000Z,2017-09-17T00:00:00.000Z,2017-09-17T00:00:00.000Z,2017-10-17T00:00:00.000Z,2017-09-23T00:00:00.000Z,2000000.00,0.0,1.0,0.0,2000000.00,2017-09-17T00:00:00.000Z,2017-12-20T00:00:00.000Z,0.0,1.0,0.0,2000000.00,Normal,Provide ground transportation to move cargo a...,"As requested by and in coordination with FEMA,...",976c56be-eb77-4435-b89b-47706ced5cfb,7cb32e10dabf23ccc24209356fbfbc34fb590b52,2024-08-27T18:04:54.403Z
36780,2016081301,Severe Storms and Flooding,Flood,4277.0,DR,SEVERE STORMS AND FLOODING,4277DRLAIAIP01,2,227839,FOS,NaN,Normal,6,LA,IAIP,INFO ANALYSIS & INFRAST PROT,Stafford Act,2017-03-24T00:00:00.000Z,None,2016-11-17T00:00:00.000Z,2017-06-30T00:00:00.000Z,2017-03-28T00:00:00.000Z,0.00,0.0,1.0,0.0,0.00,2016-11-17T00:00:00.000Z,2017-10-31T00:00:00.000Z,0.0,1.0,0.0,0.00,Normal,Amendment #2: Amend to extend end date to 6/30...,Activation of a critical infrastructure SME fr...,da350364-0a64-430f-a073-f9dcd10c8e11,9b21375ac4231cddee95e32f3f2236ac66061614,2024-08-27T18:04:54.403Z
37094,2015093004,Hurricane Joaquin,Hurricane,NaN,SU,None,PA15093004GSA01,1,205669,FOS,NaN,High,3,PA,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2016-03-23T00:00:00.000Z,2016-03-23T00:00:00.000Z,2015-10-02T00:00:00.000Z,2015-10-09T00:00:00.000Z,2016-03-23T00:00:00.000Z,-10000.00,0.0,1.0,0.0,-10000.00,2015-10-02T00:00:00.000Z,2015-10-09T00:00:00.000Z,0.0,1.0,0.0,-10000.00,Normal,Amendment 01- Amendment to change Project mana...,Pre-Declaration activation for appropriate GSA...,6043ab2a-d442-4269-af28-49f6266712e8,23c8e4d5901b5ac15ee4e4d100cca5ea5e3ca230,2024-08-27T18:04:54.403Z


In [40]:
print(df_load['incidentId'].min(), df_load['incidentId'].max())

2012082201 2025081801


In [41]:
df_load[df_load['incidentId']==2012082201]

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
40337,2012082201,Hurricane Isaac,Hurricane,4084.0,DR,HURRICANE ISAAC,4084DRFLGSA01,2,165748,FOS,NaN,Normal,4,FL,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2013-04-23T00:00:00.000Z,2013-04-23T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,2013-04-23T00:00:00.000Z,-12279.2,0.0,1.0,0.0,-12279.2,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,0.0,1.0,0.0,-12279.2,Normal,"Amendment #2: De-obligate $12,279.20 per POC/...",Post-Declaration activation for appropriate GS...,43453211-f4c2-40a9-bd65-a28bb2b8d411,c603c54b23b25a28a569ecaea4b89c4420c74c3e,2024-08-27T18:04:54.403Z
40338,2012082201,Hurricane Isaac,Hurricane,4084.0,DR,HURRICANE ISAAC,4084DRFLGSA01,1,157740,FOS,NaN,Normal,4,FL,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2012-10-31T00:00:00.000Z,2012-10-30T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,2012-10-31T00:00:00.000Z,5000.0,0.0,1.0,0.0,5000.0,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,0.0,1.0,0.0,5000.0,Normal,Amendment # 1 Request additional $5000.00 for ...,Post-Declaration activation for appropriate GS...,b908e133-f794-4112-b48c-c7f82175ede0,465ad5cb1fe3bc2cc61887af16a290311432def0,2024-08-27T18:04:54.403Z
40339,2012082201,Hurricane Isaac,Hurricane,4084.0,DR,HURRICANE ISAAC,4084DRFLGSA01,0,157223,FOS,NaN,High,4,FL,GSA,GENERAL SERVICES ADMIN.,Stafford Act,2012-10-22T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,2012-10-23T00:00:00.000Z,10000.0,0.0,1.0,0.0,10000.0,2012-10-22T00:00:00.000Z,2012-11-05T00:00:00.000Z,0.0,1.0,0.0,10000.0,Normal,Activate General Services Administration (GSA)...,Post-Declaration activation for appropriate GS...,93ee3733-615a-4929-9bf4-b2109e1e550d,162eb3cb98ba5b9132b6f939155905bf76c4e8b4,2024-08-27T18:04:54.403Z


Drop rows with no suppportFunction values as we only care about those that do have values.

In [42]:
df = df_load.dropna(subset='supportFunction')

Treat all columns as strings but preserve NaNs.

In [43]:
df = df.astype(str)
for column in df.columns:
    nan_cells = df[column].isna()
    df[column] = df[column].astype(str)
    df.loc[nan_cells, column] = np.nan
df.isna().sum()

incidentId              0
incidentName            0
incidentType            0
disasterNumber          0
declarationType         0
declarationTitle        0
maId                    0
maAmendNumber           0
actionId                0
maType                  0
supportFunction         0
priority                0
region                  0
stt                     0
agencyId                0
agency                  0
authority               0
dateReceived            0
dateRequired            0
popStartDate            0
popEndDate              0
dateObligated           0
obligationAmount        0
sttCostSharePct         0
fedCostSharePct         0
sttCostShareAmt         0
fedCostShareAmt         0
maPopStartDate          0
maPopEndDate            0
maSttCostSharePct       0
maFedCostSharePct       0
maSttCostShareAmount    0
maFedCostShareAmount    0
maPriority              0
assistanceRequested     0
statementOfWork         0
id                      0
hash                    0
lastRefresh 

Display columns in the dataset, verify they are unique

In [44]:
columns = df.columns.to_list()
print(f'There are {len(columns)} in the dataset and {len(set(columns))} are unique')
columns

There are 39 in the dataset and 39 are unique


['incidentId',
 'incidentName',
 'incidentType',
 'disasterNumber',
 'declarationType',
 'declarationTitle',
 'maId',
 'maAmendNumber',
 'actionId',
 'maType',
 'supportFunction',
 'priority',
 'region',
 'stt',
 'agencyId',
 'agency',
 'authority',
 'dateReceived',
 'dateRequired',
 'popStartDate',
 'popEndDate',
 'dateObligated',
 'obligationAmount',
 'sttCostSharePct',
 'fedCostSharePct',
 'sttCostShareAmt',
 'fedCostShareAmt',
 'maPopStartDate',
 'maPopEndDate',
 'maSttCostSharePct',
 'maFedCostSharePct',
 'maSttCostShareAmount',
 'maFedCostShareAmount',
 'maPriority',
 'assistanceRequested',
 'statementOfWork',
 'id',
 'hash',
 'lastRefresh']

Display basic description of dataframe columns

In [45]:
df.describe(include='all')

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh
count,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832,32832
unique,440,381,24,586,3,168,9862,33,32832,3,23,4,10,56,144,202,1,3190,3294,1883,2873,3269,13480,4,4,1838,13721,1870,2246,4,4,1303,13717,4,31799,8736,32832,32832,1380
top,2020030901,COVID-19 Pandemic,Hurricane,nan,DR,COVID-19 PANDEMIC,DC20030901DOD15,0,476360,FOS,0.0,High,4,FL,DOD,DEPARTMENT OF DEFENSE,Stafford Act,2024-10-12T00:00:00.000Z,None,2020-08-22T00:00:00.000Z,2021-04-30T00:00:00.000Z,2018-09-12T00:00:00.000Z,0.0,0.0,1.0,0.0,0.0,2020-08-22T00:00:00.000Z,2021-04-30T00:00:00.000Z,0.0,1.0,0.0,0.0,High,"Activate GSA to RRCC, IOF, JFO, or other teams...","As directed by and in coordination with FEMA, ...",ecbcf24e-7d9e-4ced-ae57-4871181703e1,497d57a5e4d071da1721548476b50b64a7321bdd,2024-08-27T18:24:53.439Z
freq,8997,8997,11506,5611,23439,8040,33,9582,1,22308,7677,18618,7997,3717,5131,2925,32832,102,1438,1079,655,111,4550,29228,29228,29879,4550,1079,1145,30352,30352,30803,4550,16577,41,74,1,1,27790


Print distribution of values found in specific columns

In [46]:
df['incidentName'].value_counts()

incidentName
COVID-19 Pandemic                                    8997
Hurricane Maria                                      1704
Tropical Cyclone Helene                              1501
Hurricane Florence                                   1025
Remnants of Hurricane Ida                            1001
                                                     ... 
RIV August Flooding                                     1
MT - severe storms and flooding - 5-19 - 6-3-2013       1
SD-Severe Storm - 5-31-2013                             1
AR - Severe Storms - Tornadoes - Flooding               1
R5 WI - Aug 2025 - Severe Storms and Flooding           1
Name: count, Length: 381, dtype: int64

In [47]:
df['incidentType'].value_counts()


incidentType
Hurricane              11506
Biological              8997
Tropical Storm          3106
Fire                    2466
Flood                   2066
Severe Storm            1881
Typhoon                  682
Tropical Depression      367
Coastal Storm            363
Earthquake               337
Tornado                  286
Other                    277
Severe Ice Storm         149
Volcanic Eruption        140
Dam/Levee Break           53
Mud/Landslide             48
Snowstorm                 43
Winter Storm              20
Chemical                  18
Freezing                   7
Terrorist                  7
Straight-Line Winds        6
Toxic Substances           4
Tsunami                    3
Name: count, dtype: int64

In [48]:
df['declarationType'].value_counts()

declarationType
DR    23439
SU     5611
EM     3782
Name: count, dtype: int64

In [49]:
df['declarationTitle'].value_counts()

declarationTitle
COVID-19 PANDEMIC                                                8040
None                                                             5611
HURRICANE MARIA                                                  1682
WILDFIRES                                                         829
TROPICAL STORM HELENE                                             806
                                                                 ... 
TROPICAL STORM HILARY                                               2
SEVERE STORMS, TORNADOES, FLOODING, LANDSLIDES, AND MUDSLIDES       1
SEVERE STORMS, TORNADO, AND FLOODING                                1
SEVERE STORM AND STRAIGHT-LINE WIND                                 1
SEVERE STORMS, STRAIGHT-LINE WINDS, FLOODING, AND MUDSLIDES         1
Name: count, Length: 168, dtype: int64

In [50]:
df['maType'].value_counts()

maType
FOS    22308
DFA    10484
TA        40
Name: count, dtype: int64

In [51]:
df['supportFunction'] = df['supportFunction'].astype(float).astype(int)
df['supportFunction'].value_counts()

supportFunction
0     7677
3     5716
8     3895
7     3027
5     2147
13    1621
10    1376
1     1073
11     925
4      904
6      848
12     821
2      682
9      459
18     335
19     310
20     240
17     197
16     167
14     162
15     153
21      54
22      43
Name: count, dtype: int64

In [52]:
df['agencyId'].value_counts()

agencyId
DOD         5131
GSA         1890
COE-SAD     1794
HHS         1726
EPA         1538
            ... 
OCFO           1
DOC-BIS        1
DHS-OPS        1
DOJ-USTP       1
USDAOCIO       1
Name: count, Length: 144, dtype: int64

In [53]:
df['authority'].value_counts(dropna=False)

authority
Stafford Act    32832
Name: count, dtype: int64

In [54]:
df['assistanceRequested'].value_counts()

assistanceRequested
Activate GSA to RRCC, IOF, JFO, or other teams and facilities as requested                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [55]:
df['statementOfWork'].value_counts()

statementOfWork
As directed by and in coordination with FEMA, the US Army Corps of Engineers (USACE) will provide appropriate personnel to the RRCC, IOF, JFO, or other facilities in support of disaster operations. Support may include, but is not limited to, the following: - USACE support to RRCC - USACE support to National and/or Regional IMAT - USACE modeling and/or National Hurricane Program support - USACE Local Government Liaison(s) - USACE support to ESF #15 External Affairs - USACE SME(s) - Infrastructure Systems Recovery Support Function (IS-RSF) Field Coordinator (under NDRF) - Contract or OFA audit support to ensure costs incurred under USACE MAs meet all Stafford Act and regulatory requirements This mission assignment may also fund other site-specific personnel and non-mission specific administrative and management support at a Recovery Field Office (RFO) or other USACE node locations. Those costs are not considered in this initial estimate. Costs would vary based on the magn

In [56]:
df['maPriority'].value_counts()

maPriority
High               16577
Normal             11239
Life Sustaining     2935
Lifesaving          2081
Name: count, dtype: int64

In [57]:
df['region'].value_counts()

region
4     7997
9     6152
2     5081
6     4892
3     2624
1     1635
10    1423
5     1379
7      981
8      668
Name: count, dtype: int64

Plot some information about columns and/or column combinations.

In [58]:
all_states = (df['stt'].unique())
all_states_sorted = sorted(df['stt'].unique())
regions = list(range(1,11))
agency_ids = sorted(df['agencyId'].unique())
incidents = sorted(df['incidentType'].unique())
esfs = sorted(df['supportFunction'].unique())

fig = px.scatter(df.groupby(
        ['agencyId', 'region']).size(       
    ).reset_index(
        name='count').sort_values(by='region'),
                 x = 'agencyId',
                 y = 'region',
                 size = 'count',
                 title = 'Agency by Region',
                 labels = {'agencyId' : 'Agency',
                           'region' : 'Region'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = agency_ids,
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = regions,
    tickmode = 'linear',
    dtick = 1
)

fig.update_layout(
    width = 1600
)

fig.show()

There are definitely some data quality issue with the agencyId values. Those will be addressed later.

In [59]:
fig = px.scatter(df.groupby(
        ['agencyId', 'incidentType']).size(       
    ).reset_index(
        name='count').sort_values(by='incidentType'),
                 x = 'agencyId',
                 y = 'incidentType',
                 size = 'count',
                 title = 'Agency by Incident Type',
                 labels = {'agencyId' : 'Agency',
                           'incidentType' : 'Incident Type'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = agency_ids,
    tickmode = 'linear',
    dtick = 1
)

fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = incidents,
    tickmode = 'linear',
    dtick = 1
)

fig.update_layout(
    height = 800,
    width = 1600
)

fig.show()

In [60]:
fig = px.scatter(df.groupby(
        ['supportFunction', 'incidentType']).size(       
    ).reset_index(
        name='count').sort_values(by='incidentType'),
                 x = 'supportFunction',
                 y = 'incidentType',
                 size = 'count',
                 title = 'Emergency Support Function by Incident Type',
                 labels = {'supportFunction' : 'Emergency Support Function',
                           'incidentType' : 'Incident Type'},)
fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = esfs,
    tickvals=list(range(0, 23)),

)
fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = incidents,
    tickmode = 'linear',
    dtick = 1,
)
fig.update_layout(
    height = 800,
    width = 800,
)

fig.show()

In [61]:
fig = px.scatter(df.groupby(
        ['supportFunction', 'agencyId']).size(       
    ).reset_index(
        name='count').sort_values(by='agencyId'),
                 x = 'supportFunction',
                 y = 'agencyId',
                 size = 'count',
                 title = 'Emergency Support Function by Incident Type',
                 labels = {'supportFunction' : 'Emergency Support Function',
                           'incidentType' : 'Incident Type'},)
fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = esfs,
    tickmode = 'linear',
    dtick = 1
)
fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = agency_ids,
    tickmode = 'linear',
    dtick = 1
)
fig.update_layout(
    height = 1600,
    width = 800
)

fig.show()

In [62]:
df['year'] = df['incidentId'].str[:4]
df.head()

,incidentId,incidentName,incidentType,disasterNumber,declarationType,declarationTitle,maId,maAmendNumber,actionId,maType,supportFunction,priority,region,stt,agencyId,agency,authority,dateReceived,dateRequired,popStartDate,popEndDate,dateObligated,obligationAmount,sttCostSharePct,fedCostSharePct,sttCostShareAmt,fedCostShareAmt,maPopStartDate,maPopEndDate,maSttCostSharePct,maFedCostSharePct,maSttCostShareAmount,maFedCostShareAmount,maPriority,assistanceRequested,statementOfWork,id,hash,lastRefresh,year
0,2024081901,R1_Severe Storm-Flooding_08182024,Severe Storm,3612.0,EM,"SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSL...",3612EMCTDOI-USGS01,0,476360,FOS,5,High,1,CT,DOI-USGS,DOI-USGS (GEOLOGICAL SURVEY),Stafford Act,2024-08-22T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,2024-08-23T00:00:00.000Z,20000.0,0.0,1.0,0.0,20000.0,2024-08-22T00:00:00.000Z,2024-09-04T00:00:00.000Z,0.0,1.0,0.0,20000.0,High,USGS Field measurements of flood-water heights...,"As directed by and in coordination with FEMA, ...",ecbcf24e-7d9e-4ced-ae57-4871181703e1,497d57a5e4d071da1721548476b50b64a7321bdd,2024-08-27T18:24:53.439Z,2024
1,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901USDA-APH01,0,475083,FOS,11,High,2,PR,USDA-APH,USDA-APHIS (ANIM/PLANT HEALTH,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,29184.0,0.0,1.0,0.0,29184.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,29184.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",72dc30d3-d804-4005-897f-91395be284b2,e0e65997764dc0d660e5b941a18d2f18adabaa55,2024-08-27T18:24:53.439Z,2024
2,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901DOT01,0,475067,FOS,1,High,2,PR,DOT,DOT (DEPT OF TRANSPORT-S/E),Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,25000.0,0.0,1.0,0.0,25000.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,25000.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",61a653ea-688c-4218-bef2-662e1c8b7fde,dbb5a30a3fbd2c96318ba12decb71e63ac072b76,2024-08-27T18:24:53.439Z,2024
3,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901DOJ-ATF01,0,475098,FOS,13,High,2,PR,DOJ-ATF,"DOJ-ATF (ALCOHOL,TOBACCO,FIRE",Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,73500.0,0.0,1.0,0.0,73500.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,73500.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",eeafed44-bdfa-45be-a2ec-6aeb4199043d,4f49b6e6d98634659a77d554c06b8fdeac4503cd,2024-08-27T18:24:53.439Z,2024
4,2024080901,Tropical Disturbance_TS_Ernesto,Tropical Storm,nan,SU,None,PR24080901DOE-OE01,0,475064,FOS,12,High,2,PR,DOE-OE,DOE-OE (ELEC DELIVERY / ENERG,Stafford Act,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,2024-08-09T00:00:00.000Z,124822.0,0.0,1.0,0.0,124822.0,2024-08-09T00:00:00.000Z,2024-08-23T00:00:00.000Z,0.0,1.0,0.0,124822.0,High,As directed by and in accordance with FEMA Reg...,"As directed by and in coordination with FEMA, ...",bdb7935b-5df5-4124-87d8-ddd8feb4a8e3,35d63a525dff8c4a93b83d676d303b6089c0373a,2024-08-27T18:24:53.439Z,2024


In [63]:
df['incidentType'].unique()

array(['Severe Storm', 'Tropical Storm', 'Fire', 'Hurricane', 'Flood',
       'Tornado', 'Other', 'Biological', 'Winter Storm', 'Snowstorm',
       'Volcanic Eruption', 'Coastal Storm', 'Severe Ice Storm',
       'Dam/Levee Break', 'Typhoon', 'Earthquake', 'Freezing',
       'Mud/Landslide', 'Toxic Substances', 'Chemical', 'Terrorist',
       'Tsunami', 'Tropical Depression', 'Straight-Line Winds'],
      dtype=object)

In [64]:
fig = px.scatter(df.groupby(
        ['year', 'incidentType']).size(       
    ).reset_index(
        name='count').sort_values(by='year'),
                 x = 'year',
                 y = 'incidentType',
                 size = 'count',
                 title = 'Incident Type by year',
                 labels = {'year' : 'Year',
                           'incidentType' : 'Incident Type'},)

fig.update_xaxes(
    categoryorder = 'array',
    categoryarray = sorted(df['year'].unique()),
    tickmode = 'linear',
    dtick = 1
)
fig.update_yaxes(
    categoryorder = 'array',
    categoryarray = incidents,
    tickmode = 'linear',
    dtick = 1
)
fig.update_layout(
    height = 800,
    width = 800
)
fig.show()

We want to get an idea of how many times a supportFunction value appears with others in the same incidentId. Our first attempt will be utilizing a Dependency Wheel chart from Highcharts. This code was crafted with the assistance of ChatGPT, Google Gemini, and Stack Overflow.

Note: This library does not always display properly, so the output is to a .png file instead.

In [65]:
# Create supportFunction pairs

pair_counts = {}

for incident_id, group in df.groupby("incidentId"):
    funcs = group["supportFunction"].dropna().unique()

    for a, b in combinations(sorted(funcs), 2):
        pair_counts[(a, b)] = pair_counts.get((a, b), 0) + 1

out = (
    pd.DataFrame(
        [(a, b, count) for (a, b), count in pair_counts.items()],
        columns=["supportFunction_A", "supportFunction_B", "count"]
    )
    .sort_values(["supportFunction_A", "supportFunction_B"])
    .reset_index(drop=True)
)

print(out)

results_list = out.values.tolist()

# Utilize pair counts to create the dependency wheel
cleaned = [
    {"from": str(a), "to": str(b), "weight": int(w)}
    for a, b, w in results_list
]

series = DependencyWheelSeries(
    name="Support Function Links",
    data=cleaned,
    keys=["from", "to", "weight"]
)

plot_opts = DependencyWheelOptions(
    node_padding=12,
    node_width=25
)

options = HighchartsOptions(
    chart={"type": "dependencywheel"},
    title={"text": "ESF Dependency Wheel"},
    plot_options={"dependencywheel": plot_opts},
    series=[series]
)

# Wrap in Chart object
chart = Chart.from_options(options)

chart.download_chart(filename='esf_incidentType_wheel.png')


     supportFunction_A  supportFunction_B  count
0                    0                  1     95
1                    0                  2     70
2                    0                  3    119
3                    0                  4     62
4                    0                  5     75
..                 ...                ...    ...
247                 19                 21      4
248                 19                 22      2
249                 20                 21      4
250                 20                 22      2
251                 21                 22      2

[252 rows x 3 columns]


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01\x90\x08\x02\x00\x00\x00\xfdW\x89\xcf\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x01\xec\xbd\x07p\\\xc7\x99\xef\x0b\xaf\xd6\xb6\xd6\x96\xf6\xba\xde\xfa]\xaf\xf6zk\xdf\xba\xd6U~\xb5~\xd7\xda\x85-Q\x96\xbc\\%k\x95L\xe5\x1chY\xc1V\xba2eK\xa2hYV\xf0\xd2\x92\xa8@\x91`\x04\t\x12DN\x83\x9c\x07y\x90\x07i\x00L\xce9\xe79\xb9\xfb\xb9\xa7g\x0e\x86\x83A \x08\xc9"\xd9S(\xa0\xa7O\x9f>\xdd\xff\x9e\x9a\x1f\xbe\xee\xef\xeb\xce\x83\xe4E\x14 \n\x10\x05\x88\x02D\x81\x0bX\x81\xbc\x0b\xb8\xef\xa4\xebD\x01\xa2\x00Q\x80(@\x14\x80\x04\x84\xe4C@\x14 \n\x10\x05\x88\x02\x17\xb4\x02\x04\x84\x17\xf4\xf0\x93\xce\x13\x05\x88\x02D\x01\xa2\x00\x01!\xf9\x0c\x10\x05\x88\x02D\x01\xa2\xc0\x05\xad\x00\x01\xe1\x05=\xfc\xa4\xf3D\x01\xa2\x00Q\x80(@@H>\x03D\x01\xa2\x00Q\x80(pA+@@xA\x0f?\xe9<Q\x80(@\x14 \n\x10\x10\x92\xcf\x00Q\x80(@\x14 \n\\\xd0\n\x10\x10^\xd0\xc3O:O\x14 \n\x10\x05\x88\x02\x04\x84\xe43@\x14 \n\x10\x05\x88\x02\x17\xb4\x02\x04\x84\x17\xf4\

The next plot is a circular Sankey chart using Plotly, which is not quite as successful a visualization but does appear in line. This was done with the help of Chat-GPT.

In [66]:
pairs = []
for incident, group in df.groupby("incidentId"):
    funcs = group["supportFunction"].unique()
    pairs.extend(combinations(sorted(funcs), 2))

pair_counts = Counter(pairs)

nodes = sorted(set([f for pair in pair_counts.keys() for f in pair]))
num_nodes = len(nodes)
angles = np.linspace(0, 2*np.pi, num_nodes, endpoint=False)
radius = 1
node_coords = {node: (radius*np.cos(a), radius*np.sin(a)) for node, a in zip(nodes, angles)}
node_counts = df['supportFunction'].value_counts().to_dict()
min_size = 10
max_size = 40

def size_for_count(count):
    min_count = min(node_counts.values())
    max_count = max(node_counts.values())
    if max_count == min_count:
        return (min_size + max_size) / 2
    # linear scaling
    return min_size + (count - min_count) / (max_count - min_count) * (max_size - min_size)

node_sizes = [size_for_count(node_counts[node]) for node in nodes]

max_count = max(pair_counts.values())
min_count = min(pair_counts.values())

def color_for_value(value):
    # Linear interpolation from light gray to red to black
    cmap = colors.LinearSegmentedColormap.from_list("", ["lightgray", "red", "black"])
    norm = (value - min_count) / (max_count - min_count) if max_count != min_count else 0
    rgb = cmap(norm)[:3]  # RGB tuple 0-1
    return f'rgb({int(rgb[0]*255)}, {int(rgb[1]*255)}, {int(rgb[2]*255)})'

fig = go.Figure()

# Add links
for (a, b), value in pair_counts.items():
    x0, y0 = node_coords[a]
    x1, y1 = node_coords[b]
    fig.add_trace(go.Scatter(
        x=[x0, x1],
        y=[y0, y1],
        mode='lines',
        line=dict(width=2, color=color_for_value(value)),
        hoverinfo='text',
        text=f"{a} → {b}: {value}",
        showlegend=False
    ))

# Add nodes
fig.add_trace(go.Scatter(
    x=[coord[0] for coord in node_coords.values()],
    y=[coord[1] for coord in node_coords.values()],
    mode='markers+text',
    text=nodes,
    textposition="top center",
    marker=dict(size=node_sizes, color='orange'),
    hoverinfo='text',
    showlegend=False
))

# Layout adjustments
fig.update_layout(
    title="Support Function Dependency Wheel (Links Coded Gray to Red to Black)",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    width=700,
    height=700,
    plot_bgcolor="white"
)

fig.show()


Both versions of the dependency wheel show that there are many interconnections between all Emergency Support Functions, even for the more rare values. There are strong connections between 3 and 7, 3 and 8, 3 and 13, and 7 and 13, but this is also a byproduct of these being the most frequently occuring ESFs in the dataset.